In [ ]:
!pip install google-generativeai
!pip install gradio huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 7.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 10.2 MB/s eta 0:00:00
  Created wheel for ffmpy: filename=ffmpy-0.3.2-py3-none-any.whl size=5

In [ ]:
from pathlib import Path
import google.generativeai as genai
import re
from PIL import Image
import os
#from google.colab import userdata
#os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
#GOOGLE_API_KEY = os.environ['GOOGLE_API_KEY']
#genai.configure(api_key=os.environ["GOOGLE_API_KEY"])
#or use this for personal notebook genai.configure(api_key="AIzaSyD----")

# Configuration for our Gemini Models

textgeneration_config = {
  "temperature": 0.9,
  "top_p": 1,
  "top_k": 1,
  "max_output_tokens": 2048,}


visiongeneration_config = {
  "temperature": 0.9,
  "top_p": 1,
  "top_k": 10,
  "max_output_tokens": 1024,
}

safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
]

# Two models - vision and text
textmodel = genai.GenerativeModel('gemini-1.0-pro',
                              generation_config=textgeneration_config,
                              safety_settings=safety_settings)

#imagemodel = genai.GenerativeModel('gemini-pro-vision')

visionmodel = genai.GenerativeModel(model_name="gemini-1.0-pro-vision-latest",
                              generation_config=visiongeneration_config,
                              safety_settings=safety_settings)

In [ ]:
# Utility Functions
# Convert an image to base64 string format
import base64
def img2base64(image):
    with open(image, 'rb') as img:
        encoded_string = base64.b64encode(img.read())
    return encoded_string.decode('utf-8')


# Check image format and display user messages in GUI
def user_inputs(history, txt, img):
    if not img:
        history += [(txt, None)]
        return history

    # Open the image for format verification
    try:
        with Image.open(img) as image:
            # Get image format (e.g., PNG, JPEG)
            image_format = image.format.upper()
    except (IOError, OSError):
        return history

    if image_format not in ('JPEG','JPG','PNG'):
        print(f"Warning: Unsupported image format: {image_format}")
        return history

    base64 = img2base64(img)
    data_url = f"data:image/{image_format.lower()};base64,{base64}"
    history += [(f"{txt} ![]({data_url})", None)]

In [ ]:
import gradio as gr

TITLE = """<h1 align="center">Your Personal Health Coach</h1>"""
SUBTITLE = """<h2 align="center">Upload an image of your food to knows its calories, macronutrients or ask questions about heath and exercise.</h2>"""
DES = """
<div style="text-align: center; display: flex; justify-content: center; align-items: center;">
    <span>You need to enter your FREE GEMINI KEY in the first text box to connect to Gemini Models. You can find your key here:
        <a href="https://makersuite.google.com/app/apikey">GOOGLE API KEY</a>. <br><br>
        <b> If you wish to ask a question unrelated to the image you have uploaded, just cross the image (top right corner of image) and then submit your question in the textbox.
    </span>
</div>
"""

def generate_model_response(api_key, history, text, img):
  genai.configure(api_key=api_key)
  if not img:
        text = "You are an expert nutritionist and fitness coach. You are accurate, you always stick to the facts, and never make up new facts. \
            For the questions asked by the user, answer accurately and to the point, in a friendly tone." + text
        response = textmodel.generate_content(text)

  else:
        text = "From the image uploaded by the user answer with following information: Food items in the image, \
            percentage of each macronutrient in the food in image and approximate number of calories in the food in image. If there is any additional question, answer that too." + text
        img = Image.open(img)
        response = visionmodel.generate_content([text,img])
  history += [(None, response.text)]
  return history


with gr.Blocks() as app:

  gr.HTML(TITLE)
  gr.HTML(SUBTITLE)
  gr.HTML(DES)

  api_key_box = gr.Textbox(placeholder = "Enter your GEMINI API KEY", label="Your GEMINI API KEY", type="password")

  with gr.Row():
    image_box = gr.Image(type="filepath")
    chatbot = gr.Chatbot(
      scale=3,
      height=750
    )

  text_box = gr.Textbox(
    placeholder="Ask something about the image your uploaded or ask for any health and fitness advice without uploading an image too",
    container=False,
  )

  btn = gr.Button("Submit")

  btn_clicked = btn.click(user_inputs,
                     [chatbot, text_box, image_box],
                     chatbot).then(generate_model_response,[api_key_box, chatbot, text_box, image_box], chatbot)

app.queue()
app.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://0c06c2e0748315c63f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://0c06c2e0748315c63f.gradio.live


In [ ]:
from huggingface_hub import login
login()

In [ ]:
!git clone https://huggingface.co/spaces/nimrita/YourPersonalHealthCoach


Cloning into 'YourPersonalHealthCoach'...
remote: Enumerating objects: 4, done.
remote: Total 4 (delta 0), reused 0 (delta 0), pack-reused 4
Unpacking objects: 100% (4/4), 1.29 KiB | 439.00 KiB/s, done.
